# Part I #

In [7]:
import pandas as pd
df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [12]:
n, p = df.shape[0], df.shape[1] - 1

In [13]:
import pymc as pm; import numpy as np
X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 112 seconds.


# Part II #

1. Rewrite $p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y})$ in terms of $\sigma^2$ (no longer using $\Sigma$) if $\Sigma=\sigma^2I$

\begin{align*}
p(\boldsymbol \beta |\boldsymbol\sigma^2, \mathbf{X},\mathbf{y}) = \mathcal{MVN}\left(E[\boldsymbol \beta] = (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top \mathbf{y}, \text{Var}[\boldsymbol \beta] = \boldsymbol\sigma^2 \left[\mathbf{X}^{\top} \mathbf{X} \right]^{-1} \right)\\
\end{align*}

2. What is $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

\begin{align*}
E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X}^\top \boldsymbol\Sigma^{-1} \mathbf{X})^{-1}\mathbf{X}^\top \boldsymbol\Sigma^{-1}\mathbf{y}
\end{align*}

3. What ***hyperparameters*** values (legal or illegal) would make $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

Infinite $\Sigma_\beta$ would make the prior has no effect on the posterior distribution, and thus $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$.

4. What ***hyperparameters*** values (legal or illegal) would make $E[  \mathbf{\hat y} = \mathbf{X}\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \mathbf{X}(\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

Infinite $\Sigma_\beta$ would make the prior has no effect on the posterior distribution, and thus $E[  \mathbf{\hat y} = \mathbf{X}\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \mathbf{X}(\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$.

5. What is $\text{Var}[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

\begin{align*}
Var[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = [\mathbf{X}^{\top} \boldsymbol\Sigma^{-1} \mathbf{X}]^{-1} + \boldsymbol \Sigma_\beta^{-1}
\end{align*}

# Part III #

In [20]:
import numpy as np; from scipy import stats
Psi=np.eye(p); a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)
y=stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)

with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    Sigma = pm.Deterministic('Sigma', L.dot(L.T))
    mu = pm.MvNormal('mu', mu=np.zeros(p), cov=np.eye(p)*1e-6, shape=p);
    y = pm.MvNormal('y', mu=mu, chol=L, observed=y)
    
with MNV_LKJ:  
    trace = pm.sample(n)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L, mu]


Sampling 4 chains for 1_000 tune and 918 draw iterations (4_000 + 3_672 draws total) took 352 seconds.
